# DACON - 태양광 발전량 예측 competition
***

## 1. Package Import

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import GradientBoostingRegressor
import os
from lightgbm import LGBMRegressor
import datetime

***

## 2. Load Data

In [33]:
train = pd.read_csv('/Users/kisehyun/Competition/데이콘_태양광 발전량 예측/train/train.csv')

for i in range(81) :
    globals() [f'test_{i}'] = pd.read_csv(f'/Users/kisehyun/Competition/데이콘_태양광 발전량 예측/test/{i}.csv')
    globals() [f'test_{i}']['sep_day'] = i
    
### 각 데이터를 하나의 데이터로 통합
test = pd.DataFrame()
idx_list = []
for i in range(81):
    test = pd.concat([test, globals()[f'test_{i}']], axis = 0, ignore_index = True)
    
submission = pd.read_csv('sample_submission.csv')

***

## 3. Setting


### Loss Function Definition


In [5]:
def pb_loss(true, pred, q) :
    
    L_list = []
    
    for i in range(len(true)) :
        
        if true.iloc[i] >= pred[i] :
            L = true.iloc[i] - pred[i]
            L *= q
            
        else :
            L1 = pred[i] - true.iloc[i]
            L2 = 1 - q
            L = L1 * L2
            
        L_list.append(L)
        
    return np.mean(L_list)

### custom definition to make dataset
- after 1 day

- after 2 days

In [6]:
def make_data() :
    ### 1일 후 데이터
    train_after_1 = train.loc[train.Day <= 1093]
    train_after_1['TARGET1'] = list(train.loc[train.Day >= 1].TARGET)
    
    ### 2일 후 데이터
    train_after_2 = train.loc[train.Day <= 1092]
    train_after_2['TARGET2'] = list(train.loc[train.Day >= 2].TARGET)
    
    ### after 1 day data 
    a_1day = test.loc[test.Day != 6].drop(['Day', 'Minute','sep_day','DHI', 'RH'], axis = 1)
    
    ### after 2 day data 
    a_2day = test.query('Day not in [5, 6]').drop(['Day', 'Minute','sep_day','DHI', 'RH'], axis = 1)
    
    ### after 1 day train data
    X_1 = train_after_1.drop(['Day','Minute', 'TARGET1','DHI', 'RH'], axis = 1)
    y_1 = train_after_1.TARGET1
    
    ### after 2 days train data
    X_2 = train_after_2.drop(['Day','Minute', 'TARGET2','DHI', 'RH'], axis = 1)
    y_2 = train_after_2.TARGET2
    
    ### actual values
    real_1 = test.loc[test.Day != 0].TARGET
    real_2 = test.query('Day not in [0, 1]').TARGET
    
    target = test.loc[test.Day == 6].drop(['Day', 'sep_day', 'Minute','DHI', 'RH'], axis = 1)
    
    return a_1day, a_2day, X_1, y_1, X_2, y_2, real_1, real_2, target

In [7]:
a_1day, a_2day, X_1, y_1, X_2, y_2, real_1, real_2, target = make_data()

## 4. Modeling

- 4-1. Baseline Modeling

In [32]:
"""t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'###### {qt + 1}번째 학습 및 검증 시작 ######')
    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 10000, learning_rate=0.027, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)
    lgbm.fit(X_1, y_1, eval_set = [(a_1day, real_1)], early_stopping_rounds = 1000, verbose = 0)
    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])
    print(f'best loss는 {best_loss}\n')
    t_loss.append(best_loss)
    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]
    submission.loc[submission.id.str.contains('y7'), submission.columns[1+ qt]] = lgbm_pred
lgbm_submission = submission.copy()
print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')"""

"t_loss = []\nqt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]\n\nfor qt in range(9) :\n    print(f'###### {qt + 1}번째 학습 및 검증 시작 ######')\n    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 10000, learning_rate=0.027, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)\n    lgbm.fit(X_1, y_1, eval_set = [(a_1day, real_1)], early_stopping_rounds = 1000, verbose = 0)\n    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])\n    print(f'best loss는 {best_loss}\n')\n    t_loss.append(best_loss)\n    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]\n    submission.loc[submission.id.str.contains('y7'), submission.columns[1+ qt]] = lgbm_pred\nlgbm_submission = submission.copy()\nprint(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')\nprint(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')"

In [31]:
'''
t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'###### {qt + 1}번째 학습 및 검증 시작 ######')
    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 10000, learning_rate=0.027, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)
    lgbm.fit(X_2, y_2, eval_set = [(a_2day, real_2)], early_stopping_rounds = 1000, verbose = 0)
    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])
    print(f'best loss는 {best_loss}\n')
    t_loss.append(best_loss)
    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]
    lgbm_submission.loc[lgbm_submission.id.str.contains('y8'), lgbm_submission.columns[1+ qt]] = lgbm_pred
lgbm_submission = submission.copy()
print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')'''

"\nt_loss = []\nqt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]\n\nfor qt in range(9) :\n    print(f'###### {qt + 1}번째 학습 및 검증 시작 ######')\n    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 10000, learning_rate=0.027, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)\n    lgbm.fit(X_2, y_2, eval_set = [(a_2day, real_2)], early_stopping_rounds = 1000, verbose = 0)\n    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])\n    print(f'best loss는 {best_loss}\n')\n    t_loss.append(best_loss)\n    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]\n    lgbm_submission.loc[lgbm_submission.id.str.contains('y8'), lgbm_submission.columns[1+ qt]] = lgbm_pred\nlgbm_submission = submission.copy()\nprint(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')\nprint(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')"

In [10]:
#lgbm_submission.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,3.323674,5.950682,7.430979,8.612396,9.695862,10.454627,11.074363,11.503308,12.043146
std,7.941390,13.350002,16.213885,18.454676,20.521277,21.941976,23.093581,23.853893,24.624551
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.093000,0.292361,0.722282,1.060001,1.338530,1.524804,2.046636,2.165494,4.143567
max,57.936168,76.344054,83.976837,84.113728,88.829084,92.560228,93.721818,96.195308,97.153809


In [ ]:
#lgbm_submission.to_csv('lgbm_baseline.csv', index = False)

#### it's score was bad. so, i made some features to get better score.

1. 일조시간

In [34]:
tr_times = []
for day in train.Day.unique() :
    d = train.loc[train.Day == day]
    try :
        start = d.loc[d.DHI > 0].index[0]
        end = d.loc[d.DHI > 0].index[-1]
        gap = (end - start) / 2
    except :
        gap = 0

    tr_times.append(gap)

In [35]:
test['sep'] = test.index // 48 # test 데이터의 경우 시계열 고려가 안되었기 때문에 48(하루 시간)으로 나눈 몫으로 day 구분자 생성


te_times = []
for day in test.sep.unique() :
    d = test.loc[test.sep == day]
    try :
        start = d.loc[d.DHI > 0].index[0]
        end = d.loc[d.DHI > 0].index[-1]
        gap = (end - start) / 2
    except :
        gap = 0

    te_times.append(gap)

In [36]:
train_sun_time = pd.DataFrame({'Day' : train.Day.unique(), 'sun_time' : tr_times})
test_sun_time = pd.DataFrame({'sep' : test.sep.unique(), 'sun_time' : te_times})

In [37]:
train = pd.merge(train, train_sun_time, how = 'left', on = 'Day')
test = pd.merge(test, test_sun_time, how = 'left', on = 'sep')#.drop('sep', axis = 1)

2. GHI

In [38]:
def get_d(x) :
    # 평균 기온 기준으로 계절별 평균 남중 고도 적용
    if x < 6.8 :
        return 29.5
    elif 6.8 <= x <= 20 :
        return 53
    else :
        return 76.5

In [39]:
tr_degree = train.groupby('Day')['T'].mean().reset_index(name = 'degree')
tr_degree.index.name = None
tr_degree.head()

,Day,degree
0,0,-7.979167
1,1,-6.312500
2,2,-6.479167
3,3,-5.687500
4,4,0.854167


In [40]:
te_degree = test.groupby('sep')['T'].mean().reset_index(name = 'degree')
te_degree.index.name = None
te_degree.head()

,sep,degree
0,0,4.652083
1,1,5.450000
2,2,3.414583
3,3,0.512500
4,4,-2.258333


In [41]:
tr_degree['degree'] = tr_degree.degree.apply(get_d)
te_degree['degree'] = te_degree.degree.apply(get_d)

In [42]:
train = pd.merge(train, tr_degree, how = 'left', on = 'Day')
test = pd.merge(test, te_degree, how = 'left', on = 'sep')

In [43]:
train['GHI'] = train.DNI * np.cos(train.degree) + train.DHI
test['GHI'] = test.DNI * np.cos(test.degree) + test.DHI

In [44]:
train.drop('degree', axis = 1, inplace = True)
test.drop(['degree', 'sep'], axis = 1, inplace = True)

I tried Log transformation but it was not good.

In [45]:
#train.DHI = np.log1p(train.DHI)
#test.DHI = np.log1p(test.DHI)

In [46]:
#train.DNI = np.log1p(train.DNI)
#test.DNI = np.log1p(test.DNI)

In [47]:
#train.WS = np.log1p(train.WS)
#test.WS = np.log1p(test.WS)

In [48]:
a_1day, a_2day, X_1, y_1, X_2, y_2, real_1, real_2, target = make_data()

- 4-2. 2nd Modeling

#### LGBMRegressor

In [49]:
t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'##### {qt + 1}번째 학습 및 검증 시작 #####')
    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 1000, learning_rate=0.03, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)
    lgbm.fit(X_1, y_1, eval_set = [(a_1day, real_1)], early_stopping_rounds = 1000, verbose = 0)
    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])
    print(f'best loss는 {best_loss}\n')
    t_loss.append(best_loss)
    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]
    submission.loc[submission.id.str.contains('y7'), submission.columns[1+ qt]] = lgbm_pred
lgbm_submission = submission.copy()

print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')

##### 1번째 학습 및 검증 시작 #####
best loss는 1.302995320215186

##### 2번째 학습 및 검증 시작 #####
best loss는 2.2230898578558786

##### 3번째 학습 및 검증 시작 #####
best loss는 2.687330416306145

##### 4번째 학습 및 검증 시작 #####
best loss는 2.858551858080494

##### 5번째 학습 및 검증 시작 #####
best loss는 2.7785073358665655

##### 6번째 학습 및 검증 시작 #####
best loss는 2.4554895046142753

##### 7번째 학습 및 검증 시작 #####
best loss는 1.9532143562837594

##### 8번째 학습 및 검증 시작 #####
best loss는 1.427640739460141

##### 9번째 학습 및 검증 시작 #####
best loss는 0.8120563648234905

전체 loss 평균은 2.05543
전체 loss 편차는 0.68896


In [50]:
lgbm_submission.loc[lgbm_submission.id.str.contains('y7')].describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,3888.000000,3888.000000,3888.000000,3888.000000,3888.000000,3.888000e+03,3.888000e+03,3.888000e+03,3.888000e+03
mean,6.484087,11.834539,15.185044,17.555492,19.879731,2.139597e+01,2.264152e+01,2.350213e+01,2.463045e+01
std,9.745630,16.540765,20.622315,23.389711,26.069255,2.788446e+01,2.927874e+01,3.012987e+01,3.103396e+01
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000,0.000000,0.000000,0.000000,0.000000,5.803682e-13,4.745854e-12,5.392804e-11,6.481728e-12
50%,0.223082,0.697661,1.030951,1.399471,1.552717,1.966066e+00,2.213182e+00,2.882458e+00,4.055254e+00
75%,11.159183,20.884007,28.727846,33.946843,39.496927,4.306898e+01,4.551753e+01,4.712832e+01,4.920254e+01
max,48.551494,63.402795,79.886043,81.196822,92.694826,9.301789e+01,9.458179e+01,9.707302e+01,9.780310e+01


In [51]:
t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'##### {qt + 1}번째 학습 및 검증 시작 #####')
    lgbm = LGBMRegressor(random_state = 519, objective = 'quantile', n_estimators = 1000, learning_rate=0.03, metric = 'quantile', alpha = qt_list[qt], max_depth = 4)
    lgbm.fit(X_2, y_2, eval_set = [(a_2day, real_2)], early_stopping_rounds = 1000, verbose = 0)
    best_loss = float(str(lgbm.best_score_['valid_0']).split(',')[-1][1:-3])
    print(f'best loss는 {best_loss}\n')
    t_loss.append(best_loss)
    lgbm_pred = [0 if x < 0 else x for x in lgbm.predict(target)]
    submission.loc[submission.id.str.contains('y8'), submission.columns[1+ qt]] = lgbm_pred
lgbm_submission = submission.copy()

print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')

##### 1번째 학습 및 검증 시작 #####
best loss는 1.3626250895126122

##### 2번째 학습 및 검증 시작 #####
best loss는 2.3787225765125686

##### 3번째 학습 및 검증 시작 #####
best loss는 2.872849323745674

##### 4번째 학습 및 검증 시작 #####
best loss는 3.0367286226685812

##### 5번째 학습 및 검증 시작 #####
best loss는 2.929792934353513

##### 6번째 학습 및 검증 시작 #####
best loss는 2.5743049384738574

##### 7번째 학습 및 검증 시작 #####
best loss는 2.0628176343256355

##### 8번째 학습 및 검증 시작 #####
best loss는 1.510323290345773

##### 9번째 학습 및 검증 시작 #####
best loss는 0.8526275015971662

전체 loss 평균은 2.17564
전체 loss 편차는 0.73509


In [52]:
lgbm_submission.loc[lgbm_submission.id.str.contains('y8')].describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,3888.000000,3888.000000,3888.000000,3888.000000,3888.000000,3.888000e+03,3.888000e+03,3.888000e+03,3.888000e+03
mean,5.023025,11.105847,14.766497,17.529297,20.184718,2.176260e+01,2.298863e+01,2.378018e+01,2.502499e+01
std,6.959319,15.098396,19.578004,22.904128,26.226595,2.825112e+01,2.966426e+01,3.040751e+01,3.132450e+01
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,4.912236e-12
25%,0.000000,0.000000,0.000000,0.000000,0.000000,1.313251e-10,3.292974e-12,1.106524e-11,4.912236e-12
50%,0.138199,0.581691,1.176255,1.550694,1.848206,1.842820e+00,2.270427e+00,2.904750e+00,3.999233e+00
75%,9.553232,21.529600,29.404593,35.718352,41.326017,4.369702e+01,4.595659e+01,4.699066e+01,5.011793e+01
max,22.964305,65.660935,76.591277,81.782343,95.657670,9.658606e+01,9.656857e+01,9.701681e+01,9.819879e+01


#### GradientBoostingRegressor

In [53]:
t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'##### {qt + 1}번째 학습 및 검증 시작 #####')
    gbm = GradientBoostingRegressor(random_state = 1231, loss = 'quantile', alpha = qt_list[qt], max_depth = 4)
    gbm.fit(X_1, y_1)
    gbm_pred = gbm.predict(a_1day)
    loss = pb_loss(real_1, gbm_pred, qt_list[qt])
    print(f'loss는 {loss}\n')
    t_loss.append(loss)
    gbm_pred = [0 if x < 0 else x for x in gbm.predict(target)]
    submission.loc[submission.id.str.contains('y7'), submission.columns[1+ qt]] = gbm_pred
gbm_submission = submission.copy()
print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')

##### 1번째 학습 및 검증 시작 #####
loss는 1.307221842581278

##### 2번째 학습 및 검증 시작 #####
loss는 2.233259073852083

##### 3번째 학습 및 검증 시작 #####
loss는 2.7037102980818846

##### 4번째 학습 및 검증 시작 #####
loss는 2.868915048715335

##### 5번째 학습 및 검증 시작 #####
loss는 2.8071890099413053

##### 6번째 학습 및 검증 시작 #####
loss는 2.5006752949918862

##### 7번째 학습 및 검증 시작 #####
loss는 2.014025375855133

##### 8번째 학습 및 검증 시작 #####
loss는 1.4994987358033163

##### 9번째 학습 및 검증 시작 #####
loss는 0.8640010534200248

전체 loss 평균은 2.08872
전체 loss 편차는 0.67964


In [54]:
t_loss = []
qt_list = [.09, .19, .28, .37, .48, .59, .702, .8, .9]

for qt in range(9) :
    print(f'##### {qt + 1}번째 학습 및 검증 시작 #####')
    gbm = GradientBoostingRegressor(random_state = 1231, loss = 'quantile', alpha = qt_list[qt], max_depth = 4)
    gbm.fit(X_2, y_2)
    gbm_pred = gbm.predict(a_2day)
    loss = pb_loss(real_2, gbm_pred, qt_list[qt])
    print(f'loss는 {loss}\n')
    t_loss.append(loss)
    gbm_pred = [0 if x < 0 else x for x in gbm.predict(target)]
    submission.loc[submission.id.str.contains('y8'), submission.columns[1+ qt]] = gbm_pred
gbm_submission = submission.copy()
print(f'전체 loss 평균은 {round(np.mean(t_loss), 5)}')
print(f'전체 loss 편차는 {round(np.std(t_loss), 5)}')

##### 1번째 학습 및 검증 시작 #####
loss는 1.3621095726720596

##### 2번째 학습 및 검증 시작 #####
loss는 2.3837078700186622

##### 3번째 학습 및 검증 시작 #####
loss는 2.8892350630250525

##### 4번째 학습 및 검증 시작 #####
loss는 3.0606422631312937

##### 5번째 학습 및 검증 시작 #####
loss는 2.9538994133115613

##### 6번째 학습 및 검증 시작 #####
loss는 2.611175443460016

##### 7번째 학습 및 검증 시작 #####
loss는 2.108087525001815

##### 8번째 학습 및 검증 시작 #####
loss는 1.5778061994379504

##### 9번째 학습 및 검증 시작 #####
loss는 0.9086098963501054

전체 loss 평균은 2.20614
전체 loss 편차는 0.72663


In [391]:
gbm_submission.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,6.335922,11.514949,14.905705,17.446463,19.811836,21.445360,22.725336,23.522476,24.818246
std,9.385291,16.032742,20.043526,23.070594,25.785543,27.773939,29.230340,30.117441,30.717524
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000204,0.000631,0.001084,0.001630
50%,0.148936,0.475107,1.054065,1.300678,1.790515,2.021285,2.931271,2.638427,5.500229
75%,11.301639,21.112824,28.804406,34.150701,39.684708,43.023884,45.802351,46.291108,48.841000
max,52.848297,84.035331,90.216481,87.390825,91.923439,94.654569,96.071678,97.149824,98.524096


## 5. Submission

In [55]:
submission.iloc[:, 1:] = lgbm_submission.iloc[:, 1:] * .73 + gbm_submission.iloc[:, 1:] * .27

In [56]:
zero_t = submission.loc[submission.id.str.contains('_0h|_1h|_2h|_3h|_4h|_21h|_22h|_23h')]
zero_t.iloc[:, 1:] = 0

In [57]:
submission.loc[zero_t.index] = zero_t

In [58]:
submission.describe()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
count,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000,7776.000000
mean,5.891438,11.476713,14.951592,17.504113,19.965454,21.542745,22.776332,23.665982,24.919470
std,8.674328,15.851524,20.067641,23.100607,26.051722,27.996034,29.398726,30.215727,31.132981
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.138876,0.584583,1.139847,1.359390,1.674578,1.859127,2.224759,3.084545,4.615001
75%,10.566377,21.241918,29.084958,34.788629,40.159825,43.351225,45.811862,47.008205,49.676728
max,49.808974,65.181333,78.357302,80.584780,92.882674,95.500522,96.093468,96.539954,97.864677


In [59]:
save_time = datetime.datetime.strftime(datetime.datetime.today(),'%Y%m%d_%H%M%S')

submission.to_csv(f'sun_{save_time}.csv', index=False)